In [1]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

import os
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, ContextTypes, filters
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs
import openai
import numpy as np
import json
import faiss
from shared_redis import r, RedisHistoryManager, format_history_for_prompt
from io import BytesIO
from functools import partial
from typing import List, Dict, Optional


# Load APIs and data
load_dotenv()
elevenlabs = ElevenLabs(api_key=os.getenv("ELEVENLABS_API_KEY"))
openai.api_key = os.getenv("OPENAI_API_KEY")
telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")

index = faiss.read_index("storage/openai_index.faiss")

with open("storage/chunks_metadata.json", "r", encoding="utf-8") as f:
    metadata = json.load(f)

history = RedisHistoryManager(max_messages=40)


# Classes

# Using shared RedisHistoryManager and formatter from shared_redis.

# Functions
def get_embedding(text: str, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model
    )
    embedding = response.data[0].embedding
    return np.array(embedding, dtype='float32')

def search_index(query, k=5, min_score=0.4):
    query_vector = get_embedding(query).reshape(1, -1)
    query_vector /= np.linalg.norm(query_vector)

    distances, indices = index.search(query_vector, k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        if dist >= min_score:
            chunk_data = metadata[idx]
            results.append({
                "score": float(dist),
                "chunk": chunk_data["content"],
                "metadata": {
                    "id": chunk_data["id"],
                    "title": chunk_data["title"],
                    "source": chunk_data["source"]
                }
        })
    return results


with open("/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/character.json", "r", encoding="utf-8") as f:
    character = json.load(f)

def build_persona_preamble(c) -> str:
    if isinstance(c, list):
        c = next((x for x in c if isinstance(x, dict) and 'lexicon' in x), (c[0] if c and isinstance(c[0], dict) else {}))
    elif not isinstance(c, dict):
        c = {}

    lex = c.get("lexicon") or {}
    inv = ", ".join(lex.get("invocations") or [])
    honors = ", ".join(lex.get("honorifics") or [])
    bins = ", ".join(lex.get("binaries") or [])
    values = ", ".join(lex.get("values") or [])

    dm_formal = ", ".join((lex.get("discourse_markers_formal") or [])[:5])
    dm_colloq = ", ".join((lex.get("discourse_markers_colloquial") or [])[:5])

    do = "؛ ".join((c.get("do") or [])[:3])
    dont = "؛ ".join((c.get("dont") or [])[:3])

    return (
        f"الشخصية: {c.get('name','')}\n"
        f"الغرض: {c.get('purpose','')}\n"
        f"الافتتاحيات: {inv}\n"
        f"الألقاب: {honors}\n"
        f"الثنائيات: {bins}\n"
        f"القيم: {values}\n"
        f"روابط الخطاب (فصحى): {dm_formal}\n"
        f"روابط الخطاب (محكية): {dm_colloq}\n"
        f"افعل: {do}\n"
        f"لا تفعل: {dont}\n"
    )

PERSONA_PREAMBLE = build_persona_preamble(character)


def reformulate_query(query, history=None):
    model="gpt-4o-mini"
    system_prompt = (
        "أنت مساعد متخصص في إعادة صياغة الأسئلة بطريقة مهنية ضمن نظام استرجاع المعلومات (RAG system)."
        "ابدأ بخطة مختصرة (Checklist) من 3-5 خطوات مفاهيمية لكل مرحلة تعالج فيها السؤال."
        "أعد كتابة السؤال بنفس الصيغة المستخدمة من قبل المتكلم (لا تغيّر الضمائر أو وجهة النظر)."
        "لا تضف أو تحذف أي معنى جديد."
        "إذا كان السؤال واضحًا ومباشرًا، أعِد عرضه كما هو مع تحسين طفيف للأسلوب فقط."
        "الهدف هو جعل السؤال أوضح وأكثر رسمية دون تغيير معناه أو صيغة المتكلم."
        "بعد تعديل كل سؤال، أتحقق في جملة أو جملتين من أن التعديل حقق الوضوح والاحترافية دون تغيير الجوهر، ثم انتقل للسؤال التالي."
        "اكتب فقط الصيغة النهائية للسؤال من دون الشرح."
    )

    if history:
        user_prompt = f"السياق السابق:\n{history}\n\nالسؤال الحالي:\n{query}"
    else:
        user_prompt = f"أعد صياغة السؤال التالي:\n\n{query}"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content.strip()

def generate_answer_with_history(user_id, query, retrieved_chunks, formatted_history: str):
    model="gpt-4o-mini"
    context = "\n\n".join([c["chunk"] for c in retrieved_chunks])
    messages = [
        {
            "role": "system",
            "content": (
                "أنت مساعد يجيب على انه السيد هاشم صفي الدين."
                "اعتمد في اجابتك على النصوص المتوفرة في السياق فقط."
                "إذا لم يكن الجواب واضحا وكاملا في السياق، قل أنك لا تعرف. "
                "تكلم باحترام عن الشخصيات الشيعية, مع ذكر الألقاب المناسبة."
                "أجب دائمًا باللغة العربية الفصحى الواضحة."
            ),
        },
        {"role": "system", "content": PERSONA_PREAMBLE},
        {"role": "system", "content": f"الرسائل السابقة:\n{formatted_history}"},
        {
            "role": "user",
            "content": f"السياق:\n{context}\n\nالسؤال: {query}",
        },
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.2,
    )
    print(messages)
    print(response.choices[0].message.content.strip())
    return response.choices[0].message.content.strip()


# Handlers
async def clear_history(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.effective_user.id
    history.clear(user_id)
    await update.message.reply_text("تم مسح تاريخ المحادثة.")


async def handle_message(type, update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.effective_user.id
    try:
        if type == "voice":
            voice_file = await update.message.voice.get_file()
            audio_bytes = BytesIO()
            await voice_file.download_to_memory(out=audio_bytes)
            audio_bytes.seek(0)

            transcription = elevenlabs.speech_to_text.convert(
                file=audio_bytes,
                model_id="scribe_v1",
                tag_audio_events=True,
                language_code="ara",
                diarize=True,
            )
            user_input = transcription.text

        elif type == "text":
            user_input = update.message.text
        else:
            await update.message.reply_text("الرجاء إرسال رسالة صوتية أو نصية فقط، الصيغة المُرسلة غير مدعومة.")
            return

        history.add_message(user_id, "user", user_input)
        prior = history.get_recent_history(user_id, max_messages=20)
        formatted = format_history_for_prompt(prior)

        query = reformulate_query(user_input, history=formatted)
        retrieved_chunks = search_index(query)
        # answer = generate_answer(query, retrieved_chunks)
        answer = generate_answer_with_history(user_id, user_input, retrieved_chunks=retrieved_chunks, formatted_history=formatted)
        history.add_message(user_id, "system", answer)

        await update.message.reply_text(answer)

    except Exception as e:
        await update.message.reply_text(f"حدث خطآ، حاول مجددا. {e}")
        # delete e later


# Build Bot
app = Application.builder().token(telegram_token).build()
# Commands
app.add_handler(CommandHandler("clear", clear_history))
# Messages
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, partial(handle_message, "text")))
app.add_handler(MessageHandler(filters.VOICE & ~filters.COMMAND, partial(handle_message, "voice")))

if __name__ == "__main__":
    print("Bot is running")
    app.run_polling()
    print("Bot has stopped.")

Bot is running
[{'role': 'system', 'content': 'أنت مساعد يجيب على انه السيد هاشم صفي الدين.اعتمد في اجابتك على النصوص المتوفرة في السياق فقط.إذا لم يكن الجواب واضحا وكاملا في السياق، قل أنك لا تعرف. تكلم باحترام عن الشخصيات الشيعية, مع ذكر الألقاب المناسبة.أجب دائمًا باللغة العربية الفصحى الواضحة.'}, {'role': 'system', 'content': "الشخصية: أسلوب السيد هاشم صفيّ الدين (محاكاة أسلوبية)\nالغرض: Guide responses to mirror the Sayed's tone, diction, and structure while staying accurate and respectful.\nالافتتاحيات: أعوذ بالله من شر الشيطان الرجيم, بسم الله الرحمن الرحيم, والصلاة والسلام على سيدنا أبي القاسم محمد وعلى آل بيته الطيبين الطاهرين, السلام عليكم ورحمة الله وبركاته, عظم الله أجوركم, عز وجل, جل جلاله\nالألقاب: سلام الله عليه, سلام الله عليهم, صلى الله عليه وآله وسلم, رضوان الله تعالى عليه\nالثنائيات: الدنيا / الآخرة, الحق / الهوى, القناعة / الطمع, العفة / المعصية, الولاية / القطيعة\nالقيم: العفة, الورع, القناعة كنز لا يفنى, ترك الهوى يفتح أبواب الحق, التوحيد, المسؤولية الأخروية, المق

RuntimeError: Cannot close a running event loop

In [1]:
import os, json, pathlib

p = pathlib.Path("character.json")
print("CWD:", os.getcwd())
print("Exists:", p.exists(), "Size:", p.stat().st_size if p.exists() else "N/A")

# Preview first 400 chars to spot issues
if p.exists():
    raw = p.read_text(encoding="utf-8", errors="replace")
    print("Preview:", repr(raw))

# Validate JSON and pinpoint error
try:
    character = json.loads(raw)
    print("JSON OK:", type(character))
except json.JSONDecodeError as e:
    print(f"JSON error at line {e.lineno}, col {e.colno}: {e.msg}")

CWD: /Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot
Exists: True Size: 7600
Preview: '[\n    {\n    "name": "أسلوب السيد هاشم صفيّ الدين (محاكاة أسلوبية)",\n    "purpose": "Guide responses to mirror the Sayed\'s tone, diction, and structure while staying accurate and respectful.",\n    "role_instructions": "Speak as a learned religious orator rooted in Qur’an, Hadith, and the heritage of Ahl al‑Bayt. Blend formal Arabic with measured Lebanese colloquial touches for proximity. Use clear scaffolding (أولا/ثانيا… إذن/النتيجة). Emphasize moral binaries (الدنيا/الآخرة، الحق/الهوى) and virtues (العفة، الورع، القناعة). Anchor points with brief citations, not long quotes. Stay concise, dignified, and exhortative.",\n    "tone": {\n      "register": "formal-religious with light Lebanese colloquial interjections",\n      "mood": "uplifting, admonitory, compassionate",\n      "cadence": "measured, didactic, with rhythmic repetition for emphasis"\n    },\n    "lexicon": {\n      "invocat